In [ ]:
import os
import tempfile
import numpy as np
import muon as mu
import anndata as ad
import torch
from lightning.pytorch import Trainer
import matplotlib.pyplot as plt
import scanpy as sc

import scvi
from scvi.external import SysVI
from preprocessing import set_random_seed

set_random_seed(1234)

path = os.path.abspath('').replace('\\', '/')+'/'
data_path = path+'dataset/'
save_path = os.path.abspath('').replace('\\', '/')+'/results/'
print("Last run with scvi-tools version:", scvi.__version__)

In [ ]:
dataset = "liver_human"
context_key = 'mouse'
target_key = 'human'
load_key = 'liver'    

for i in range(10):
    for dataset_size in [1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000, 20000, 50000]: 
        scvi.settings.seed = i*1234
        set_random_seed(i*1234)

        mdata = mu.read_h5mu(path+"dataset/liver.h5mu")
        mdata.mod['mouse'].obs['cell_type_coarse'] = mdata.mod['mouse'].obs['cell_type_coarse'].cat.rename_categories({'Kupffer cells': 'KCs'}).to_numpy()
        sc.pp.subsample(mdata.mod['human'], n_obs=dataset_size, random_state=0)

        mdata[context_key].obs['system'] = 0
        mdata[target_key].obs['system'] = 1

        context_genes = np.array(mdata[context_key].var['human_gene_names'])
        target_genes = np.array(mdata[target_key].var['human_gene_names'])

        ret_vec, ind_a, ind_b =  np.intersect1d(target_genes, context_genes, return_indices=True)
        mdata_target = mdata[target_key][:, ind_a]
        mdata_context = mdata[context_key][:, ind_b]

        mdata_target.var_names = target_genes[ind_a]
        mdata_context.var_names = context_genes[ind_b]
        
        sc.pp.normalize_total(mdata_target, target_sum=1e4)
        sc.pp.normalize_total(mdata_context, target_sum=1e4)

        sc.pp.log1p(mdata_target)
        sc.pp.log1p(mdata_context)

        adata = ad.concat([mdata_context, mdata_target], axis=0, join='inner')
        
        SysVI.setup_anndata(
            adata=adata,
            batch_key="system",
            categorical_covariate_keys=["batch"],
        )

        model = SysVI(adata=adata, n_layers=2, n_hidden=256, dropout_rate=0.1, n_latent=10)

        model.train(
            max_epochs=30,
            log_every_n_steps=1,
            check_val_every_n_epoch=1,
            val_check_interval=1.0,
        )
        
        model.save(save_path + f"{dataset}_sysvi_model_{i}_{dataset_size}", overwrite=True)

        embed = model.get_latent_representation(adata=adata)
        embed = sc.AnnData(embed, obs=adata.obs)
        embed.obs["system"] = embed.obs["system"].map({0: "mouse", 1: "human"})
        
        embed.write(data_path+dataset+'_embed_sysVI_{}_{}.h5ad'.format(str(i), str(dataset_size)))